The goal of this exercise is to suggest to Instacart customers items that they might want to purchase based on past orders. The point is not to create an award-winning algorithm, but to place yourself in the shoes of a data science consultant. 

1. Data Exploration    
    1. What initial insights can you get from a first exploration of the dataset?  
    2. Do some variables seem to have more importance than other? What transformations might be needed?  
2. Prediction    
    1. Describe your overall approach: how did you formulate the problem to make recommendations?  
    2. What model did you choose? Why?  
    3. How did you assess performance of your model? Which metrics seemed particularly important to you?  
    4. Please present your results and model performance  
    5. How would you suggest delivering the recommendations to your client?  
3. Insights and Next Steps    
    1. What insights can you extract from your analysis?  
    2. What are some of the things you would suggest as next steps for your client?  

# Loading libraries

In [210]:
#data processing libraries
import pandas as pd
import numpy as np

#Visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

#warning and other libraries
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

# Initial thoughts

1. Important to note that I have a RAM limitation of 8Gb and time constraints of 15 hours for this problem. Due to this, I am ruling out building ML models which tends to take longer time to train - Neural Nets, Adaboost etc.
2. I suspect (to be verified later) non-linear relations and I know that LightGBM and CatBoost models generally perform well on such structured data-sets.  
3. Steps I have decided to follow - 
    1. First I will go through Exploratory Data Analysis - This has been done through the Instacart-EDA notebook.
    2. Carry out feature engineering
    3. Predict reorder for the test data-set
    4. Build the recommendation engine
    5. Document inferences and results

In [211]:
#Loading all the data-sets
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train_cap.csv')
order_products_test = pd.read_csv('order_products__test_cap.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

# Feature Engineering

## User Features

In [212]:
orders.set_index('order_id', inplace = True, drop = False)
priors = pd.merge(order_products_prior, orders, how = 'left', on = 'order_id')
priors = priors.append(order_products_train, ignore_index=True)
priors.head()

,add_to_cart_order,days_since_prior_order,eval_set,order_dow,order_hour_of_day,order_id,order_number,product_id,reordered,user_id
0,1,8.00,prior,5.00,9.00,2,3.00,33120,1,202279.00
1,2,8.00,prior,5.00,9.00,2,3.00,28985,1,202279.00
2,3,8.00,prior,5.00,9.00,2,3.00,9327,0,202279.00
3,4,8.00,prior,5.00,9.00,2,3.00,45918,1,202279.00
4,5,8.00,prior,5.00,9.00,2,3.00,30035,0,202279.00


In [213]:
users = pd.DataFrame()
users['total_user'] = priors.groupby('product_id').size()
users['all_users'] = priors.groupby('product_id')['user_id'].apply(set)
users['distinct_users_per_product'] = users.all_users.map(len)

users_features = priors.groupby("user_id").agg({'reordered': {'user_reorder_ratio': 'mean'}})
users_features.columns = users_features.columns.droplevel(0)
users_features = users_features.reset_index()

users_features = users_features.merge(priors.groupby(["user_id", "order_id"]).size().reset_index().drop("order_id", axis=1).groupby("user_id").agg([np.sum, np.mean, np.std]), how='inner', left_on='user_id',
                                         right_on='user_id')
users_features.columns = ["user_id", "user_reorder_ratio","user_basket_sum", "user_basket_mean", "user_basket_std"]

In [214]:
user_features_more = pd.DataFrame()
user_features_more['user_avg_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
user_features_more['user_number_of_orders'] = orders.groupby('user_id').size().astype(np.int16)
user_features_more['user_total_items'] = priors.groupby('user_id').size().astype(np.int16)
user_features_more['user_all_products'] = priors.groupby('user_id')['product_id'].apply(set)
#user_features_more['user_total_unique_items'] = user_features_more.user_all_products.map(len).astype(np.int16)
user_features_more['user_avg_per_cart'] = (user_features_more.user_total_items / user_features_more.user_number_of_orders).astype(np.float32)

In [215]:
user_features_more.reset_index(level=0, inplace=True)

In [216]:
users_features = users_features.join(user_features_more, how='inner', on='user_id', lsuffix='_left', rsuffix='_right')

In [217]:
users_features = users_features.drop(['user_id_left','user_id_right'], axis=1)
users_features.head()

,user_id,user_reorder_ratio,user_basket_sum,user_basket_mean,user_basket_std,user_avg_days_between_orders,user_number_of_orders,user_total_items,user_all_products,user_avg_per_cart
0,1.00,0.69,59,5.90,1.52,16.29,15,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",13.00
1,2.00,0.48,195,13.93,5.72,12.00,13,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",6.77
2,3.00,0.62,88,7.33,2.10,17.00,6,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",3.00
3,4.00,0.06,18,3.60,2.07,11.50,5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",7.40
4,5.00,0.38,37,9.25,3.10,13.33,4,14,"{40992, 27521, 20323, 48679, 8424, 45007, 2190...",3.50


## Product features

In [218]:
product_features = pd.DataFrame()

product_features['product_orders_count'] = order_products_prior.groupby(
    order_products_prior.product_id).size().astype(np.int32)

product_features['product_reorders_count'] = order_products_prior['reordered'].groupby(
    order_products_prior.product_id).sum().astype(np.float32)

product_features['product_reorder_rate'] = (product_features.product_reorders_count / product_features.product_orders_count).astype(np.float32)

#Merging products, departments and aisles indexing files
products_departments_aisles = pd.merge(left =pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')

product_features = product_features.merge(products_departments_aisles[['product_id','aisle_id', 'department_id']] , how='inner', left_on='product_id',
                                         right_on='product_id') 

In [219]:
product_features.head()

,product_id,product_orders_count,product_reorders_count,product_reorder_rate,aisle_id,department_id
0,1,1852,1136.00,0.61,61,19
1,2,90,12.00,0.13,104,13
2,3,277,203.00,0.73,94,7
3,4,329,147.00,0.45,38,1
4,5,15,9.00,0.60,5,13


# Users - Products

these features were made with direct references due to lack of time

In [220]:
priors["date"] = priors.iloc[::-1]['days_since_prior_order'].cumsum()[::-1].shift(-1).fillna(0)
max_group = priors["order_number"].max()
priors["order_number_reverse"] = max_group - priors["order_number"]

In [221]:
order_stat = priors.groupby('order_id').agg({'order_id': 'size'}).rename(columns={'order_id': 'order_size'}).reset_index()
priors = pd.merge(priors, order_stat, on='order_id')
priors['add_to_cart_order_inverted'] = priors.order_size - priors.add_to_cart_order
priors['add_to_cart_order_relative'] = priors.add_to_cart_order / priors.order_size

users_products = priors.groupby(["user_id", "product_id"]). \
        agg({'reordered': {'users_prod_nb_reordered': "size"}, \
             'add_to_cart_order': {'users_prod_add_to_cart_order_mean': "mean"}, \
             'add_to_cart_order_relative': {'users_prod_add_to_cart_order_relative_mean': "mean"}, \
             'add_to_cart_order_inverted': {'users_prod_add_to_cart_order_inverted_mean': "mean"}, \
             'order_number_reverse': {'users_prod_last_order_number': "min", 'users_prod_first_order_number': "max"}, \
             'date': {'users_prod_last_order_date': "min", 'users_prod_first_date_number': "max"}, \
            })

users_products.columns = users_products.columns.droplevel(0)
users_products = users_products.reset_index()

In [222]:
users_products.head()

,user_id,product_id,users_prod_nb_reordered,users_prod_add_to_cart_order_mean,users_prod_add_to_cart_order_relative_mean,users_prod_add_to_cart_order_inverted_mean,users_prod_last_order_number,users_prod_first_order_number,users_prod_last_order_date,users_prod_first_date_number
0,1.00,196,10,1.40,0.25,4.50,89.00,98.00,0.00,294585470.00
1,1.00,10258,9,3.33,0.56,2.67,89.00,97.00,5278374.00,294585414.00
2,1.00,10326,1,5.00,0.62,3.00,94.00,94.00,294585358.00,294585358.00
3,1.00,12427,10,3.30,0.54,2.60,89.00,98.00,0.00,294585442.00
4,1.00,13032,3,6.33,0.96,0.33,89.00,97.00,85860799.00,282947554.00


# Preparing data for training

In [223]:
dataTrain = priors.join(users_features, how='left', on='user_id', lsuffix='_left', rsuffix='_right')
del priors
del aisles
del departments
del products
dataTrain = dataTrain.drop(columns=['user_id_right'])
dataTrain.rename(columns={'user_id_left':'user_id'}, inplace=True)
dataTrain = dataTrain.join(product_features, how='left', on='product_id', lsuffix='_left', rsuffix='_right')
dataTrain = dataTrain.drop(columns=['product_id_right'])
dataTrain.rename(columns={'product_id_left':'product_id'}, inplace=True)
dataTrain = pd.merge(dataTrain, users_products,  how='left', left_on=['user_id','product_id'], right_on = ['user_id','product_id'])

In [224]:
dataTrain.head(10)

,add_to_cart_order,days_since_prior_order,eval_set,order_dow,order_hour_of_day,order_id,order_number,product_id,reordered,user_id,...,aisle_id,department_id,users_prod_nb_reordered,users_prod_add_to_cart_order_mean,users_prod_add_to_cart_order_relative_mean,users_prod_add_to_cart_order_inverted_mean,users_prod_last_order_number,users_prod_first_order_number,users_prod_last_order_date,users_prod_first_date_number
0,1,8.00,prior,5.00,9.00,2,3.00,33120,1,202279.00,...,112.00,3.00,5.00,2.00,0.19,8.60,91.00,98.00,0.00,337079945.00
1,2,8.00,prior,5.00,9.00,2,3.00,28985,1,202279.00,...,25.00,11.00,5.00,3.20,0.30,7.40,93.00,98.00,0.00,337079937.00
2,3,8.00,prior,5.00,9.00,2,3.00,9327,0,202279.00,...,120.00,16.00,1.00,3.00,0.33,6.00,96.00,96.00,337079929.00,337079929.00
3,4,8.00,prior,5.00,9.00,2,3.00,45918,1,202279.00,...,91.00,16.00,5.00,4.80,0.42,7.20,92.00,97.00,60358730.00,337079921.00
4,5,8.00,prior,5.00,9.00,2,3.00,30035,0,202279.00,...,83.00,4.00,3.00,4.67,0.42,7.67,92.00,96.00,60358748.00,337079913.00
5,6,8.00,prior,5.00,9.00,2,3.00,17794,1,202279.00,...,43.00,3.00,7.00,3.57,0.34,7.57,92.00,98.00,0.00,337079905.00
6,7,8.00,prior,5.00,9.00,2,3.00,40141,1,202279.00,...,78.00,19.00,5.00,5.80,0.57,4.40,93.00,97.00,35729888.00,337079897.00
7,8,8.00,prior,5.00,9.00,2,3.00,1819,1,202279.00,...,131.00,9.00,2.00,9.50,0.90,1.00,96.00,97.00,189404156.00,337079889.00
8,9,8.00,prior,5.00,9.00,2,3.00,43668,0,202279.00,...,90.00,7.00,3.00,6.67,0.67,3.67,93.00,96.00,60358721.00,337079881.00
9,1,12.00,prior,5.00,17.00,3,16.00,33754,1,205970.00,...,8.00,3.00,17.00,5.06,0.39,8.88,74.00,98.00,19486442.00,337079869.00


In [225]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score,recall_score,cohen_kappa_score

In [226]:
cols = [
 #'order_id',
 'product_id',
 'add_to_cart_order',
 'user_id',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 #'date',
 #'order_number_reverse',
 'order_size',
 #'add_to_cart_order_inverted',
 'add_to_cart_order_relative',
 'user_reorder_ratio',
 'user_basket_sum',
 'user_basket_mean',
 #'user_basket_std',
 'user_avg_days_between_orders',
 'user_number_of_orders',
 'user_total_items',
 #'user_total_unique_items',
 'user_avg_per_cart',
 'product_orders_count',
 'product_reorders_count',
 'product_reorder_rate',
 'aisle_id',
 'department_id',
 'users_prod_nb_reordered',
 'users_prod_add_to_cart_order_mean',
 'users_prod_add_to_cart_order_relative_mean',
 #'users_prod_add_to_cart_order_inverted_mean',
       ]

In [227]:
#Plotting Correlation between features

correlation = dataTrain[cols].corr()
matrix_cols = correlation.columns.tolist()
corr_array  = np.array(correlation)
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix",
                        height = 720,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

# Sampling

In [271]:
#Taking out 1M samples randomly for faster processing
dataTrain = dataTrain.sample(100000)

In [272]:
dataTrain = dataTrainN.copy()

# Normalization - MinMaxScaler

In [273]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [274]:
target_col = ['reordered']

X = dataTrain[cols]
Y = dataTrain[target_col]

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = .10, random_state = 111)

col_names = train_X.columns.tolist()
scaler = MinMaxScaler()
train_X[cols] = scaler.fit_transform(train_X[cols])
test_X[cols] = scaler.fit_transform(test_X[cols])

In [275]:
#function to print and plot basic graphs for each modeling method.
def reorder_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y, cols, cf, threshold_plot) :
    
    #fit the mdoel
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    #checking if coefficients or features
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    print (algorithm)
    print ("\n Classification report : \n",classification_report(testing_y,predictions))
    print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #ploting the confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not Reordered","Reordered"],
                        y = ["Not Reordered","Reordered"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    #plot coeffs
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    #subplot for ROC curve
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)

# Hyperparameter tuning - GridSearch

In [127]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6],
        'n_estimators': [50, 100, 500],
        }

In [33]:
#XGBoost - learning rate has to be low, low max_depth
xgb = XGBClassifier(base_score=0.5, 
                    booster='gbtree', 
                    learning_rate=0.01, 
                    objective='binary:logistic',
                    silent=True, 
                    nthread=1)

folds = 3
param_comb = 6

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

random_search.fit(X, Y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed: 26.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  18 out of  18 | elapsed: 26.3min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x1a2644e830>,
          error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=6, n_jobs=4,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5, 6], 'n_estimators': [50, 100, 500]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [35]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([111.22785036, 151.46831918, 735.88597727, 704.91762622,
        83.59443005,  44.20087854]), 'std_fit_time': array([ 0.72103544, 10.91831622, 12.59572929, 66.18947375,  2.05667753,
        0.78659745]), 'mean_score_time': array([1.13114595, 1.66118765, 5.44025167, 8.19592579, 0.65885091,
       0.64079301]), 'std_score_time': array([0.02541445, 0.17790966, 0.43846879, 1.3970204 , 0.01671362,
       0.04197208]), 'param_subsample': masked_array(data=[0.8, 1.0, 0.8, 0.8, 1.0, 1.0],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[100, 100, 500, 500, 50, 50],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 5, 10, 5, 5, 5],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'par

In [276]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=1.5, learning_rate=0.01,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=500, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

reorder_prediction(xgc, X, test_X, Y, test_Y, cols,"features",threshold_plot = True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

 Classification report : 
               precision    recall  f1-score   support

           0       0.94      0.74      0.83     40856
           1       0.84      0.97      0.90     59144

   micro avg       0.88      0.88      0.88    100000
   macro avg       0.89      0.85      0.87    100000
weighted avg       0.88      0.88      0.87    100000

Accuracy   Score :  0.87505
Area under curve :  0.8545396819307188 

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3           -      ]

